In [ ]:
# https://medium.com/analytics-vidhya/predicting-the-next-hindi-words-de58541fbbcf

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/NLP_Project/premchand.tsv', sep = '\t')

In [ ]:
df['Type of Work'].value_counts()

कहानियाँ    157
उपन्यास      10
Name: Type of Work, dtype: int64

In [ ]:
#For hindi 
data_h = " ".join(df['Text'].to_list())
data_h = data_h.replace('\n', " ")
data_h = data_h[:100000]

In [ ]:
len(data_h.split())

20639

In [ ]:
import numpy as np

In [ ]:
len(np.unique(data.split()))

11763

In [ ]:
len((data_h.split()))

1402326

In [ ]:
len(data_h)

6793650

In [ ]:
data = data_h

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# encode 2 words -> 1 word
sequences = list()
for i in range(4, len(encoded)):
	sequence = encoded[i-4:i+2]
	sequences.append(sequence)

Vocabulary Size: 9220


In [ ]:
#For english
path = '/content/drive/MyDrive/NLP_Project/1661-0.txt'
data = open(path).read().lower()
print('corpus length:', len(data))

corpus length: 581533


In [ ]:
data = data[:180000]+data_h

's right\nforefinger, and a bulge on the right side of his top-hat to show where\nhe has secreted his stethoscope, i must be dull, indeed, if i do not\npronounce him to be an active member of the medical profession.”\n\ni could not help laughing at the ease with which he explained his\nprocess of deduction. “when i hear you give your reasons,” i remarked,\n“the thing always appears to me to be so ridiculously simple that i\ncould easily do it myself, though at each successive instance of your\nreasoning i am baffled until you explain your process. and yet i\nbelieve that my eyes are as good as yours.”\n\n“quite so,” he answered, lighting a cigarette, and throwing himself\ndown into an armchair. “you see, but you do not observe. the\ndistinction is clear. for example, you have frequently seen the steps\nwhich lead up from the hall to this room.”\n\n“frequently.”\n\n“how often?”\n\n“well, some hundreds of times.”\n\n“then how many are there?”\n\n“how many? i don’t know.”\n\n“quite so! yo

In [ ]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
import numpy as np


In [ ]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
    # generate a fixed number of words
	for _ in range(n_words):
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		yhat = np.argmax(model.predict(encoded), axis = -1)
		print("yhat", yhat)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [ ]:
import pickle

# saving
with open('/content/drive/MyDrive/NLP_Project/tokenizer_english.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# encode 2 words -> 1 word
sequences = list()
for i in range(4, len(encoded)):
	sequence = encoded[i-4:i+2]
	sequences.append(sequence)

Vocabulary Size: 9220


In [ ]:
print('Total Sequences: %d' % len(sequences))
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Total Sequences: 55397
Max Sequence Length: 6


In [ ]:
# test set - (x (5), y (1-5))  [10,000, 10,000]
# evaluate (x, pred_y, actual_y){
#      if synonyms(pred_y, actual_y) - match   
# wordnet, nlp/ stanza 
# }

In [ ]:
# loss - match/mismatch (match/synonymous/mismatch)

In [ ]:
# 

In [ ]:
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]

#print(X)
y = to_categorical(y, num_classes=vocab_size)
#print(y)
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(300, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
# model = Sequential()
# model.add(LSTM(300, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(300))
# model.add(Dropout(0.2))
# model.add(Dense(Y_modified.shape[1], activation='softmax'))

In [ ]:
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=10, verbose=2)
# evaluate model

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 5, 10)             92200     
                                                                 
 lstm_6 (LSTM)               (None, 5, 300)            373200    
                                                                 
 dropout_6 (Dropout)         (None, 5, 300)            0         
                                                                 
 lstm_7 (LSTM)               (None, 300)               721200    
                                                                 
 dropout_7 (Dropout)         (None, 300)               0         
                                                                 
 dense_3 (Dense)             (None, 9220)              2775220   
                                                                 
Total params: 3,961,820
Trainable params: 3,961,820
No

In [ ]:
keras.load('/content/drive/MyDrive/NLP_Project/hindi_f_english_keras_prediction.h5')

In [ ]:
print(generate_seq(model, tokenizer, max_length-1, 'सितारे दिन में', 2))
print(generate_seq(model, tokenizer, max_length-1, 'कुछ लोगों को', 4))
print(generate_seq(model, tokenizer, max_length-1, 'साल का था मैंने गाड़ी चलानी ', 2))
print(generate_seq(model, tokenizer, max_length-1, 'आपसे मिलकर', 2))
print(generate_seq(model, tokenizer, max_length-1, 'मेरा नाम ', 2))

सितारे दिन में बार एक
कुछ लोगों को भी तरह न खिसकाया
साल का था मैंने गाड़ी चलानी  के तरफ
आपसे मिलकर एक दिल
मेरा नाम  न रहे


In [ ]:
print(generate_seq(model, tokenizer, max_length-1, 'सितारे दिन में', 2))
print(generate_seq(model, tokenizer, max_length-1, 'कुछ लोगों को', 4))
print(generate_seq(model, tokenizer, max_length-1, 'साल का था मैंने गाड़ी चलानी ', 2))
print(generate_seq(model, tokenizer, max_length-1, 'आपसे मिलकर', 2))
print(generate_seq(model, tokenizer, max_length-1, 'मेरा नाम ', 10))

सितारे दिन में और तैयारियॉँ
कुछ लोगों को थोड़ी निराशा हुई। यों
साल का था मैंने गाड़ी चलानी  ने हँसकर
आपसे मिलकर की अखाड़े
मेरा नाम  और मैं बातें सकता मिलेगा दरवाजे की सुन्दरता तकवे उठी।


In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/NLP_Project/english_keras_prediction.h5')

In [ ]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	input_array = [in_text]
	input_probs = [1]
	for _ in range(n_words):
		results = []
		results_prob = []
		# print(input_array, "input_arrrr")
		for index in range(len(input_array)): 
			j = input_array[index]
			# print(j, "jjjjj")
			prob_j = input_probs[index]
			encoded = tokenizer.texts_to_sequences([j])[0]
			# pre-pad sequences to a fixed length
			encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
			probs = model.predict(encoded)
			# print(probs)
			yhat = np.argsort(probs, axis=1)[:,-5:]

			# print("yhat", yhat)
			# map predicted word index to word
			for i in yhat[0]:
				# print(i)
				out_word = ''
				for word, index in tokenizer.word_index.items():
					if index == i:
						out_word = word
						break
				# append to input
				copy = j + ' '+ out_word
				# j += ' ' + out_word
				results.append(copy)
				results_prob.append(probs[0][i]*prob_j)
			# print(results_prob)

		max_probs = np.argsort([results_prob], axis=1)[:,-5:]
		input_array = []
		input_probs = []
		# print(max_probs, "max_p")
		for i in max_probs[0]:
			input_array.append(results[i])
			input_probs.append(results_prob[i])
	 
	return input_array

In [ ]:
print(generate_seq(model, tokenizer, 5, 'The king and the queen ', 4))

['The king and the queen  beamed ” “then i', 'The king and the queen  beamed ” “no the', 'The king and the queen  beamed ” “then ”', 'The king and the queen  beamed to be a', 'The king and the queen  beamed ” said holmes']
